In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

data = pd.read_csv("boston.csv")
data.dropna(inplace=True)

median_price = np.median(data["MEDV"])
data["MEDV_Class"] = np.where(data["MEDV"] >= median_price, 1, 0) 

scaler = StandardScaler()
X = scaler.fit_transform(data.drop(columns=["MEDV", "MEDV_Class"]))  
y_binary = data["MEDV_Class"].values  
y_multi = pd.qcut(data["MEDV"], q=3, labels=[0, 1, 2]).astype(int)  

print("\n Розбиття даних на навчальні та тестові підгрупи (Hold-Out)")
X_train_bin, X_test_bin, y_train_bin, y_test_bin = train_test_split(X, y_binary, test_size=0.2, random_state=42)
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(X, y_multi, test_size=0.2, random_state=42)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

print("\n Побудова моделей логістичної регресії")

log_reg_bin = LogisticRegression(max_iter=1000)
log_reg_bin.fit(X_train_bin, y_train_bin)
y_pred_bin = log_reg_bin.predict(X_test_bin)

log_reg_multi = LogisticRegression(max_iter=1000)
log_reg_multi.fit(X_train_multi, y_train_multi)
y_pred_multi = log_reg_multi.predict(X_test_multi)

accuracy_bin = accuracy_score(y_test_bin, y_pred_bin)
accuracy_multi = accuracy_score(y_test_multi, y_pred_multi)

print(f"\n Точність бінарної моделі: {accuracy_bin:.4f}")
print(f" Точність множинної моделі: {accuracy_multi:.4f}")

print("\n Classification Report для бінарної моделі:\n", classification_report(y_test_bin, y_pred_bin))
print("\n Classification Report для множинної моделі:\n", classification_report(y_test_multi, y_pred_multi))

print("\n Підбір оптимальних параметрів моделей")
param_grid = [
    {"penalty": ["l1"], "C": [0.1, 1, 10], "solver": ["liblinear"]},
    {"penalty": ["l2"], "C": [0.1, 1, 10], "solver": ["liblinear", "lbfgs"]}
]

grid_bin = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=kf)
grid_bin.fit(X_train_bin, y_train_bin)
best_params_bin = grid_bin.best_params_

grid_multi = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=kf)
grid_multi.fit(X_train_multi, y_train_multi)
best_params_multi = grid_multi.best_params_

print("\n Оптимальні параметри для бінарної моделі:", best_params_bin)
print(" Оптимальні параметри для множинної моделі:", best_params_multi)


log_reg_bin_opt = LogisticRegression(**best_params_bin, max_iter=1000)
log_reg_bin_opt.fit(X_train_bin, y_train_bin)
y_pred_bin_opt = log_reg_bin_opt.predict(X_test_bin)

log_reg_multi_opt = LogisticRegression(**best_params_multi, max_iter=1000)
log_reg_multi_opt.fit(X_train_multi, y_train_multi)
y_pred_multi_opt = log_reg_multi_opt.predict(X_test_multi)


accuracy_bin_opt = accuracy_score(y_test_bin, y_pred_bin_opt)
accuracy_multi_opt = accuracy_score(y_test_multi, y_pred_multi_opt)

print(f"\n Точність бінарної моделі після оптимізації: {accuracy_bin_opt:.4f}")
print(f" Точність множинної моделі після оптимізації: {accuracy_multi_opt:.4f}")
if accuracy_bin_opt > accuracy_multi_opt:
    print("\n Найкраща модель: Бінарна логістична регресія")
else:
    print("\n Найкраща модель: Множинна логістична регресія")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.heatmap(confusion_matrix(y_test_bin, y_pred_bin_opt), annot=True, cmap='Blues', fmt='d')
plt.xlabel("Прогноз")
plt.ylabel("Реальне значення")
plt.title("Матриця помилок (Бінарна)")

plt.subplot(1, 2, 2)
sns.heatmap(confusion_matrix(y_test_multi, y_pred_multi_opt), annot=True, cmap='Oranges', fmt='d')
plt.xlabel("Прогноз")
plt.ylabel("Реальне значення")
plt.title("Матриця помилок (Множинна)")

plt.show()